In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

In [ ]:

# Define a simple kernel to add two arrays
mod = SourceModule("""
__global__ void add_arrays(float *a, float *b, float *result, int n)
{
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < n)
        result[idx] = a[idx] + b[idx];
}
""")

In [ ]:

# Use Python lists instead of NumPy arrays
a = [1.0, 2.0, 3.0, 4.0, 5.0]
b = [10.0, 20.0, 30.0, 40.0, 50.0]
n = len(a)

# Convert Python lists to byte arrays for GPU processing
a_bytes = bytearray(struct.pack('%sf' % len(a), *a))
b_bytes = bytearray(struct.pack('%sf' % len(b), *b))
result_bytes = bytearray(n * 4)  # 4 bytes per float



In [ ]:
# Allocate GPU memory
a_gpu = cuda.mem_alloc(len(a_bytes))
b_gpu = cuda.mem_alloc(len(b_bytes))
result_gpu = cuda.mem_alloc(len(result_bytes))


In [ ]:

# Transfer data to the GPU
cuda.memcpy_htod(a_gpu, a_bytes)
cuda.memcpy_htod(b_gpu, b_bytes)


In [ ]:

# Execute the kernel
add_arrays = mod.get_function("add_arrays")
add_arrays(a_gpu, b_gpu, result_gpu, np.int32(n), block=(n, 1, 1))

In [ ]:
# Fetch the result back to CPU
cuda.memcpy_dtoh(result_bytes, result_gpu)
result = struct.unpack('%sf' % n, result_bytes)

print("Result:", result)